In [ ]:
#!/usr/bin/env python3

from dotenv import load_dotenv
import os
# from pprint import pprint
from tikapi import TikAPI, ValidationException, ResponseException
from rich.pretty import pprint
from rich.console import Console
console = Console()
import requests

load_dotenv('.env.local')
token = os.environ.get("tik_api_key")
api = TikAPI(token)

def p(object):
    pprint(object, max_depth=2, console=console)

def downloadfile(name,url):
    name=name+".mp4"
    r=requests.get(url)
    print("****Connected****")
    f=open(name,'wb')
    print("Donloading.....")
    for chunk in r.iter_content(chunk_size=255): 
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
            # print(chunk)
    print("Done")
    f.close()


try:
    response = api.public.hashtag(
        name="gym"
    )

    hashtagId = response.json()['challengeInfo']['challenge']['id']

    response = api.public.hashtag(
        id=hashtagId
    )


    # pprint(response.json())
    items = response.json()['itemList']
    # for item in items:
    #     p('NEW ITEM')
    #     p(item)

    item = items[6]
    pprint(item)
    p(item['textExtra'])
    downloadAddr = item['video']['downloadAddr']
    name = item['video']['id']
    downloadfile(name, downloadAddr)
    response.save_video(downloadAddr, name+'.mp4')
    

    # print(response)



# iteration
    # while(response):
    #     cursor = response.json().get('cursor')
    #     print("Getting next items ", cursor)
    #     response = response.next_items()

except ValidationException as e:
    print(e, e.field)

except ResponseException as e:
    print(e, e.response.status_code)

In [ ]:
import ffmpeg

(
    ffmpeg
    .input(name+'.mp4')
    .output(name+'.mp3')
    .run()
)


In [ ]:
import whisper
from whisper.utils import exact_div, format_timestamp, optional_int, optional_float, str2bool, write_txt, write_vtt, write_srt


model = whisper.load_model("small")
result = model.transcribe(name+".mp3")
# print(result["text"])
# f= open(name+'.txt',"w+")
# f.write(result['text'])
# f.close()
with open(os.path.join(name + ".txt"), "w", encoding="utf-8") as txt:
            write_txt(result["segments"], file=txt)


